# attention

In [ ]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1115394


Let's have a look at the first 1000 characters:

In [ ]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



Extracting the characters from the dataset:

In [3]:
chars = sorted(list(set(text)))
print("".join(chars))
vocab_size = len(chars)
print(f"{vocab_size=}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65


How to tokenize the input text? We can create a mapping from characters to integers.

In [4]:
stoi = { s: i for i, s in enumerate(chars) }
itos = { i: s for i, s in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("hello world"))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]


In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Prepare the train/val splits:

In [6]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Transformers are trained on chunks of data of size `block_size`. As we are interested in the sequence of characters, we need to sample `block_size+1` characters to obtain `block_size` sequences.

In [7]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Context: {context}, target: {target}")

Context: tensor([18]), target: 47
Context: tensor([18, 47]), target: 56
Context: tensor([18, 47, 56]), target: 57
Context: tensor([18, 47, 56, 57]), target: 58
Context: tensor([18, 47, 56, 57, 58]), target: 1
Context: tensor([18, 47, 56, 57, 58,  1]), target: 15
Context: tensor([18, 47, 56, 57, 58,  1, 15]), target: 47
Context: tensor([18, 47, 56, 57, 58,  1, 15, 47]), target: 58


A batch size of 4, with a block size of 8, allows to generate 32 examples.

In [9]:
torch.manual_seed(0)
bs = 4
block_size = 8 # context length for predictions

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (bs, )) # get random offsets
    x = torch.stack([data[i:i + block_size] for i in ix]) # stack by row
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

for b in range(bs): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"When input is {context}, target is {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 43,  1, 45, 47, 60, 43, 57],
        [51,  1, 58, 46, 43,  1, 63, 53],
        [39, 50, 50,  1, 63, 53, 59,  1],
        [53, 61, 52,  1, 57, 53, 59, 50]])
targets:
torch.Size([4, 8])
tensor([[43,  1, 45, 47, 60, 43, 57,  1],
        [ 1, 58, 46, 43,  1, 63, 53, 59],
        [50, 50,  1, 63, 53, 59,  1, 46],
        [61, 52,  1, 57, 53, 59, 50,  1]])
When input is tensor([46]), target is 43
When input is tensor([46, 43]), target is 1
When input is tensor([46, 43,  1]), target is 45
When input is tensor([46, 43,  1, 45]), target is 47
When input is tensor([46, 43,  1, 45, 47]), target is 60
When input is tensor([46, 43,  1, 45, 47, 60]), target is 43
When input is tensor([46, 43,  1, 45, 47, 60, 43]), target is 57
When input is tensor([46, 43,  1, 45, 47, 60, 43, 57]), target is 1
When input is tensor([51]), target is 1
When input is tensor([51,  1]), target is 58
When input is tensor([51,  1, 58]), target is 46
When input is tensor([51,  1, 58, 4

Let's feed a minibatch to the bigram language model.

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

In [16]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    

    def forward(self, idx, targets=None):
        # There are vocab_size channels
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)

        if targets is None:
            loss = None
        else:
            # cross entropy expects the data to be reshaped
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)

            # we want to measure the quality of these logits
            # the negative log likelihood does that
            loss = F.cross_entropy(logits, targets)

        return logits, loss


    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)

        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss) # -ln(1/65) = 4.17

torch.Size([32, 65]) tensor(4.9966, grad_fn=<NllLossBackward0>)


Generate from the model:

In [18]:
idx = torch.zeros((1, 1), dtype=torch.long) # a single character
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


dOozjwY rCmM!co&iKiCw lZzHIRl-ragsD:;kDOlZlPz!in?ZzlWLys;PAeZoWn;?nTxU
B!wTchAxxhvU'j,RcBF'JTjt$PUV.


We are feeding the model with the entire context but use the last character only to make the next prediction.

Let's train the model.

In [19]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [25]:
bs = 32
for steps in range(1000):
    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
idx = torch.zeros((1, 1), dtype=torch.long) # a single character
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))

2.8433220386505127


Shy-
FPEdr, le3Q3KaNMPotpNTzS ? k!stond suGx banoUPharSTh LQhilf! gh inXfaVhttIsoumy?
ot s.QThe?MNdqYxZB!ryr zmi33ZBOFDsiges;atswar g,
DSLOF' frengefll:

me artw$Rco,'stias onmarBAUFYOMNO-PL:qfrMET:-
TckDof!n !

V, t, hi?j,gsS!Gden! bcUFeikngnd aHRGAme;
ARULa:QISio ivid INIVLp.A, IOF.


FIspe g

3e


Tokens are not talking to each other but we see an improvement.